In [ ]:
%%capture
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases #Package for database reading.
!pip install mne #The MNE Package is installed
FILEID = "1-bPsREsUCOiJHzIqi8DQrfSjTAf5VAW_"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.
!dir

#Import

In [ ]:
from google.colab.drive import mount
from gcpds.databases.BCI_Competition_IV import Dataset_2a
import numpy as np
from MI_EEG_ClassMeth.Preprocessing import butterworth_digital_filter, ica_artiact_removal, spherical_spline_surface_laplacian
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr
from mne.channels import make_standard_montage
from mne import create_info
import os
from time import time
from pickle import dump

#Mount Drive

In [ ]:
mount("/content/drive", force_remount=True)

Mounted at /content/drive


#Get Data

os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/none_none/training/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/none_none/evaluation/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/ica_none/training/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/ica_none/evaluation/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/none_sl/training/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/none_sl/evaluation/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/ica_sl/training/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/ica_sl/evaluation/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/labels/training/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/labels/evaluation/')

os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/results/ica/training/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/results/ica/evaluation/')

In [ ]:
data_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/'
results_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/results/'
seed = 23
db = Dataset_2a('/content/drive/Shareddrives/GCPDS/databases/BCI_Competition_IV/dataset_2a')
subjects = np.arange(db.metadata['subjects']) + 1
fs = db.metadata['sampling_rate']
eeg_ch_names = db.metadata['channels'][:-3]
montage = make_standard_montage(db.metadata['montage'])
info = create_info(eeg_ch_names, fs, len(eeg_ch_names)*['eeg'])
info.set_montage(montage)
t_mi = np.cast['uint16'](np.array([2, 6])*fs)

f_bank = np.asarray([[8,12],[12,15],[15,20],[18,40]])
vwt = np.asarray([[0.5, 3.5]])
tf_repr = TimeFrequencyRpr(sfreq=fs, f_bank=f_bank, vwt=vwt)

# ------------------------------------------------------------------------------
t_i = time()
for mode in ['training', 'evaluation']:
  y_dir = os.path.join(data_dir , 'labels/' + mode + '/')

  for sbj in subjects:
    db.load_subject(sbj, mode=mode)
    X, y = db.get_data()
    X = X*1e6 #uV

    X = butterworth_digital_filter(X, N=5, Wn=1, btype='highpass', fs=fs)
    X = X[:,:,t_mi[0]:t_mi[1]]
    X_eeg = X[:,:-3,:]
    A = X[:,-3:,:]

    np.save(y_dir + 'sbj' + str(sbj), y)
    
    for ica in ['none_', 'ica_']:
      if ica == 'ica_':
        X_clean, idx_noise_trials, W_unmixing, rho = ica_artiact_removal(X_eeg, A, seed=seed)
        with open(results_dir + 'ica/' + mode + '/' + 'sbj' + str(sbj) + '.txt', 'wb') as f:
          dump({'idx_noise_trials':idx_noise_trials,
                'W_unmixing':W_unmixing,
                'rho':rho}, f)
      else:
        X_clean = X_eeg

      for sl in ['none', 'sl']:       
        if sl == 'sl':
          X_sl = spherical_spline_surface_laplacian(X_clean, info)
        else:
          X_sl = X_clean

        X_wt = tf_repr.transform(X_sl)  

        X_dir = os.path.join(data_dir , ica+sl + '/' + mode + '/')
        np.save(X_dir + 'sbj' + str(sbj), X_wt)

t_f = time() - t_i
print('time:',round(t_f/60, 2),'m')

/usr/local/lib/python3.7/dist-packages/mne/io/edf/edf.py:1047: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/usr/local/lib/python3.7/dist-packages/mne/io/edf/edf.py:1047: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/usr/local/lib/python3.7/dist-packages/mne/io/edf/edf.py:1047: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/usr/local/lib/python3.7/dist-packages/mne/io/edf/edf.py:1047: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/usr

time: 41.98 m
